## Apex - PROD

In [8]:
Import-Module AWS.Tools.Common
Import-Module AWS.Tools.DynamoDBv2
Import-Module AWS.Tools.RDS

In [7]:
$RegexGroup             = 'AccountNumber'
$RegexGroupAccountName  = 'AccountName'
$RegexAccountNumber     = "(?<$RegexGroup>\d+):db.+$"
$RegexAccountNumberDDb  = "(?<$RegexGroup>\d+):table.+$" 
$RegexAccountName       = "(?<PreFix>DevOps-)(?<$RegexGroupAccountName>.+)"
$Profiles               = Get-AWSCredentials -ListProfileDetail | Where-Object { $_.ProfileName -match 'DevOps-Apex-PROD' }
$Regions                = "eu-central-1" , "eu-west-2" , "eu-west-1", "us-east-1"

function Get-AWSDBInformation {
    [CmdletBinding()]
    param (
        $Profiles,
        $Regions
    )
    
    begin {

        $Summary    = @()
        
    }
    
    process {

        $Summary    = @()
        foreach ($Profile in $Profiles) {

            foreach ($Region in $Regions) {
                
                $RDSInstances   = Get-RDSDBInstance -ProfileName $Profile.ProfileName -Region $Region
                $ddbTables      = Get-DDBTableList -ProfileName $Profile.ProfileName -Region $Region

                switch ([string]::IsNullOrEmpty($RDSInstances)) {
                    $true { 
                        if ($ddbTables) {
                            
                            $Table = $ddbTables | Select-Object -First 1
                            $ddbTable = Get-DDBTable -TableName $Table -ProfileName $Profile.ProfileName -Region $Region
                            $AccountID  = (($ddbTable.TableArn | Select-String -Pattern $RegexAccountNumberDDb).Matches.Groups | Where-Object { $_.Name -eq $RegexGroup }).Value
                        }else {

                            $AccountID = 0
                        }
                    }
                    $false {

                        $RDSInstance   = $RDSInstances | Select-Object -First 1
                        $AccountID  = (($RDSInstance.DBInstanceArn | Select-String -Pattern $RegexAccountNumber).Matches.Groups | Where-Object { $_.Name -eq $RegexGroup }).Value
                    }
                    default {

                        $AccountID = 0
                    }
                }

                $Sum = [PSCustomObject]@{

                    Account             = (($Profile.ProfileName | select-string -Pattern $RegexAccountName).Matches.Groups | Where-Object { $_.Name -eq $RegexGroupAccountName }).Value
                    AccountID           = $AccountID
                    Region              = $Region
                    RDSInstances        = ($RDSInstances.DBInstanceIdentifier | Measure-Object).Count
                    DynamoDBTables      = $ddbTables.Count
                    
                }

                $condition = $Sum.RDSInstances + $Sum.DynamoDBTables
                if ($condition -gt 0) {
                    $Summary += $Sum
                }
            }

        }
        
    }
    
    end {

        $Summary
        
    }
}

$Summary    = Get-AWSDBInformation $Profiles $Regions
$Summary    | Format-Table -AutoSize


Account   AccountID    Region    RDSInstances DynamoDBTables
-------   ---------    ------    ------------ --------------
Apex-PROD 463854377715 eu-west-2            0              8



In [9]:
$ddbBackupList  = Get-DDBBackupList -ProfileName $Profiles.ProfileName
$ddbTableList   = Get-DDBTableList -ProfileName $Profiles.ProfileName

$ddbTables  = @()
foreach ($ddbTable in $ddbTableList) {
    
    $Table = Get-DDBTable -TableName $ddbTable -ProfileName $Profiles.ProfileName

    if ($ddbBackupList) {
        
        $BackupAvailable = $ddbBackupList.TableName.Contains("$($Table.TableName)")
    }
    else {

        $BackupAvailable = $false
    }
    

    $Sum = [PSCustomObject]@{

        TableName           = $Table.TableName
        TableBackup         = $BackupAvailable
        CreationDateTime    = $Table.CreationDateTime
        ItemCount           = $Table.ItemCount
        # TableSizeBytes      = $Table.TableSizeBytes
        TableSizeMB         = "$([math]::round($($Table.TableSizeBytes /1MB), 0)) MB"
        TableStatus         = $Table.TableStatus
        # TableId             = $Table.TableId
        # KeySchema           = $Table.KeySchema
        # GlobalSecondaryIndexes = $Table.GlobalSecondaryIndexes
        
    }
    $ddbTables += $Sum
}
$ddbTables | Format-Table -AutoSize


TableName                  TableBackup CreationDateTime      ItemCount TableSizeMB TableStatus
---------                  ----------- ----------------      --------- ----------- -----------
prod-accommodation               False 10/20/2023 1:11:55 PM    151547 28 MB       ACTIVE
prod-apex_product_extras         False 2/2/2023 4:43:28 PM     5071428 993 MB      ACTIVE
prod-apex_translations           False 5/16/2024 10:06:42 AM      7743 1 MB        ACTIVE
prod-availability                False 10/20/2023 1:11:55 PM   3883867 2617 MB     ACTIVE
prod-bookability                 False 10/20/2023 1:11:55 PM 117145542 31130 MB    ACTIVE
prod-novasol_currencies          False 2/2/2023 4:43:28 PM          38 0 MB        ACTIVE
prod-novasol_product_cache       False 10/20/2023 1:11:55 PM    213318 194 MB      ACTIVE
terraform-locks                   True 7/8/2022 6:20:39 PM          89 0 MB        ACTIVE



In [11]:
Get-DDBBackupList -ProfileName $Profiles.ProfileName | Format-Table -AutoSize

Error: (1,32): error CS1002: ; expected
(1,32): error CS1056: Unexpected character '$'